# 1 - Data Collection

In this section, we fetch data from Uniswap V3 subgraph, and store them in json for further processing.

In [1]:
# Standard Library
import datetime as dt
import glob
import json
import os
from pprint import pprint

# Third Party Library
import numpy as np
import pandas as pd
from flatdict import FlatDict
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

# Local Folder Library
from pyammanalysis.graphql_helper import run_query
from pyammanalysis.util import *

In [2]:
# refetch setting - if True, rerun GraphQL queries
refetch = True
if refetch:
    transport = AIOHTTPTransport(url=UNISWAP_V3_SUBGRAPH_URL)

# create folder if needed
for folder in [DATA_PATH, DATA_TOKEN_DAY_PATH, DATA_POOL_DAY_PATH]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# config
config = read_yaml("../config.yaml")
token_dict = config["tokens"]
token_addr_dict = config["token_addr"]  # mapping from symbol to addr
whitelisted_symbols = np.sort(
    np.concatenate([i for i in FlatDict(token_dict).itervalues()])
)

# address-related config
# addresses in `config.yaml` follow EIP-55: Mixed-case checksum address encoding
# enforce lower case by `str.lower()`
sym2addr = lambda symbol: config["token_addr"][
    symbol
].lower()  # mapping from symbol to addr
addr2sym = lambda addr: {v.lower(): k for k, v in config["token_addr"].items()}[
    addr
]  # mapping from addr to symbol
whitelisted_addresses = np.array(
    [i.lower() for i in FlatDict(token_addr_dict).itervalues()]
)

## Uniswap V3 Global Data
For now we only fetch the newest pool count and TVL.

In [ ]:
CURRENT_GLOBAL_DATA_QUERY = """
{
    factory(id: "0x1F98431c8aD98523631AE4a59f267346ea31F984" ) {
        poolCount
        totalValueLockedUSD
    }
}
"""

GLOBAL_DATA_PATH = os.path.join(DATA_PATH, "globalData.json")

if refetch:
    global_data = run_query(UNISWAP_V3_SUBGRAPH_URL, CURRENT_GLOBAL_DATA_QUERY)["data"][
        "factory"
    ]
    with open(GLOBAL_DATA_PATH, "w") as f:
        json.dump(global_data, f, indent=4)
else:
    with open(GLOBAL_DATA_PATH, "r") as f:
        global_data = json.load(f)

pprint(global_data)

## Tokens
The subsections [Token Data](#token-data) and [Token Day Time Series](#token-day-time-series) should be run in batch.
### Token Data

In [ ]:
# get top 30 tokens by TVL
TOP_TOKENS_QUERY = """
{
    tokens(first: 30, orderBy: totalValueLockedUSD, orderDirection: desc) {
        id
        symbol
        name
        totalValueLockedUSD
    }
}
"""

TOKENS_DF_PATH = os.path.join(DATA_PATH, "tokens_df.csv")

if refetch:
    top_tokens = run_query(UNISWAP_V3_SUBGRAPH_URL, TOP_TOKENS_QUERY)["data"]
    tokens_df = pd.DataFrame.from_dict(top_tokens["tokens"])
else:
    tokens_df = pd.read_csv(TOKENS_DF_PATH)

# set column dtype
tokens_df = tokens_df.astype(
    {"id": str, "symbol": str, "name": str, "totalValueLockedUSD": np.float64}
)

tokens_df.head()

### Token Day Time Series
Ref: https://github.com/Uniswap/v3-info/blob/770a05dc1a191cf229432ebc43c1f2ceb3666e3b/src/data/tokens/chartData.ts#L14

In [ ]:
def get_token_key(symbol: str, addr: str) -> str:
    """
    Generates a key for a token.
    The uniqueness of the key is guaranteed by the address,
    but the symbol is also prefixed for readability.
    """
    # return f"{symbol}_{addr}"
    return symbol

In [ ]:
TOKEN_DAY_TIME_SERIES = """
    query tokenDayDatas($startTime: Int!, $skip: Int!, $address: String!) {
        tokenDayDatas(
            first: 1000
            skip: $skip
            where: { token: $address, date_gt: $startTime }
            orderBy: date
            orderDirection: asc
            subgraphError: allow
        ) {
            date
            volumeUSD
            totalValueLockedUSD
        }
    }
"""

In [ ]:
fetch_token_error = []


async def fetch_token_chart_data(
    address: str, symbol: str, transport: AIOHTTPTransport = transport
):
    error = False
    skip = 0
    all_found = False
    result = {"tokenDayDatas": []}

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
        execute_timeout=45,  # USDC takes 40s, WETH sometimes fails
    ) as session:
        params = {"address": address, "startTime": START_TIMESTAMP, "skip": skip}
        try:
            while not all_found:
                temp = await session.execute(
                    gql(TOKEN_DAY_TIME_SERIES), variable_values=params
                )
                skip += 1000
                if len(temp["tokenDayDatas"]) < 1000 or error:
                    all_found = True
                if temp:
                    # concat the lists
                    result["tokenDayDatas"] = (
                        result["tokenDayDatas"] + temp["tokenDayDatas"]
                    )
        except Exception as e:
            print(e)
            error = True
            fetch_token_error.append(address)

    if not error:
        with open(
            f"{DATA_TOKEN_DAY_PATH}/{get_token_key(symbol, address)}.json",
            "w",
        ) as f:
            json.dump(result, f, indent=4)

In [ ]:
fetch_token_error = []

if refetch:
    # remove existing content in the out folder
    for f in glob.glob(DATA_TOKEN_DAY_PATH + "/*"):
        os.remove(f)

    # fetch token day data for each token
    for i, row in tokens_df.iterrows():
        await fetch_token_chart_data(
            row["id"],
            row["symbol"],
        )

    print(fetch_token_error)

In [ ]:
# reads token day datas from json
token_day_df = pd.DataFrame(columns=["date"])
token_names = []

for f in os.listdir(DATA_TOKEN_DAY_PATH):
    fullname = os.fsdecode(f)

    # not a rigorous check
    with open(os.path.join(DATA_TOKEN_DAY_PATH, fullname), "r") as file:
        token_day_datas = json.load(file)

    # parse dict as df
    temp = pd.DataFrame.from_dict(token_day_datas["tokenDayDatas"]).astype(
        {"volumeUSD": np.float64, "totalValueLockedUSD": np.float64}
    )

    # prefix columns (except "date") with token name
    cols = temp.columns[~temp.columns.isin(["date"])]
    token_name = fullname.split(os.sep)[-1].split(".")[0]
    token_names.append(token_name)
    temp.rename(columns=dict(zip(cols, token_name + "_" + cols)), inplace=True)

    # outer join: union of items on "date"
    token_day_df = pd.merge(token_day_df, temp, how="outer", on=["date"])

# sort by "date"
token_day_df = token_day_df.sort_values(by="date").reset_index(drop="index")

token_day_df.head()

In [ ]:
# ["date"]: int -> date (in "YYYY-MM-DD")
token_day_df["timestamp"] = token_day_df["date"]  # keep timestamp in a new col
token_day_df["date"] = token_day_df["date"].map(dt.date.fromtimestamp)

token_day_df.head()

In [ ]:
# only consider tokens with track record >= 180 days
token_day_notna = token_day_df.notna().sum()
cols_to_drop = list(token_day_notna[token_day_notna < 180].index.values)
tokens_to_drop = list(set([x.split("_")[0] for x in cols_to_drop]))

# drop cols
tokens_df = tokens_df[~tokens_df["symbol"].isin(tokens_to_drop)].reset_index()
token_day_df.drop(columns=cols_to_drop, inplace=True)

print(tokens_to_drop)

In [ ]:
token_day_df.info()

In [ ]:
# inspect number of days elapsed
d0 = token_day_df["date"].iloc[0]
d1 = token_day_df["date"].iloc[-1]
print(f"{d0} to {d1} has {(d1 - d0).days} days")

In [ ]:
if refetch:
    tokens_df.to_csv(TOKENS_DF_PATH, index=False)
    token_day_df.to_csv(os.path.join(DATA_PATH, "token_day_df.csv"), index=False)

## Pools
The subsections [Pool Data](#pool-data) and [Pool Day Time Series](#pool-day-time-series) should be run in batch.
### Pool Data

In [3]:
def get_pool_key(symbol0: str, symbol1: str, fee_tier: int) -> str:
    """
    Generates a key for a pool.
    `token0`, `token1` and `feeTier` together uniquely define a pool.
    But using symbol instead of token address involve a risk.
    """
    return f"{symbol0}_{symbol1}_{fee_tier}"

In [4]:
# get top 50 pools by TVL (but only analyze top 20)
TOP_POOLS_QUERY = """
{
    pools(first: 50, orderBy: totalValueLockedUSD, orderDirection: desc) {
        id
        token0 {
            id
            symbol
        }
        token1 {
            id
            symbol
        }
        feeTier
        totalValueLockedUSD
    } 
}
"""

POOLS_DF_PATH = os.path.join(DATA_PATH, "pools_df.csv")

if refetch:
    top_pools = run_query(UNISWAP_V3_SUBGRAPH_URL, TOP_POOLS_QUERY)["data"]
    pools_df = pd.json_normalize(top_pools["pools"])
    pools_df.to_csv(POOLS_DF_PATH, index=False)
else:
    pools_df = pd.read_csv(POOLS_DF_PATH)

# set column dtype
pools_df = pools_df.astype(
    {
        "id": str,
        "feeTier": int,
        "totalValueLockedUSD": np.float64,
        "token0.id": str,
        "token0.symbol": str,
        "token1.id": str,
        "token1.symbol": str,
    }
)

# only analyze top 20
pools_df = pools_df.iloc[:20]

pools_df.head()

,id,feeTier,totalValueLockedUSD,token0.id,token0.symbol,token1.id,token1.symbol
0,0xa850478adaace4c08fc61de44d8cf3b64f359bec,500,1.288763e+10,0x12b32f10a499bf40db334efe04226cca00bf2d9b,UMIIE,0x5ed60a121159481675bad3e648ba4c89753e056f,UMIIE2
1,0x5777d92f208679db4b9778590fa3cab3ac9e2168,100,6.485969e+08,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC
2,0x6c6bc977e13df9b0de53b251522280bb72383700,500,4.332220e+08,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC
3,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,3000,3.218534e+08,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH
4,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,500,1.938243e+08,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH


In [5]:
# whitelist a pool if both its token0 and token1 are whitelisted
is_whitelisted_pool = pools_df["token0.id"].isin(whitelisted_addresses) & pools_df[
    "token1.id"
].isin(whitelisted_addresses)
pools_df = pools_df[is_whitelisted_pool]

# add name
pools_df["name"] = pools_df.apply(
    lambda x: get_pool_key(
        addr2sym(x["token0.id"]), addr2sym(x["token1.id"]), x["feeTier"]
    ),
    axis=1,
)

pools_df.head()

,id,feeTier,totalValueLockedUSD,token0.id,token0.symbol,token1.id,token1.symbol,name
1,0x5777d92f208679db4b9778590fa3cab3ac9e2168,100,6.485969e+08,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,DAI_USDC_100
2,0x6c6bc977e13df9b0de53b251522280bb72383700,500,4.332220e+08,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,DAI_USDC_500
3,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,3000,3.218534e+08,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USDC_WETH_3000
4,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,500,1.938243e+08,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,USDC_WETH_500
5,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,3000,1.526420e+08,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,WBTC_WETH_3000


In [6]:
pools_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 1 to 18
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   13 non-null     object 
 1   feeTier              13 non-null     int32  
 2   totalValueLockedUSD  13 non-null     float64
 3   token0.id            13 non-null     object 
 4   token0.symbol        13 non-null     object 
 5   token1.id            13 non-null     object 
 6   token1.symbol        13 non-null     object 
 7   name                 13 non-null     object 
dtypes: float64(1), int32(1), object(6)
memory usage: 884.0+ bytes


### Pool Day Time Series
Ref: https://github.com/Uniswap/v3-info/blob/770a05dc1a191cf229432ebc43c1f2ceb3666e3b/src/data/pools/chartData.ts#L14

In [7]:
POOL_DAY_TIME_SERIES = """
    query poolDayDatas($startTime: Int!, $skip: Int!, $address: String!) {
        poolDayDatas(
            first: 1000
            skip: $skip
            where: { pool: $address, date_gt: $startTime }
            orderBy: date
            orderDirection: asc
            subgraphError: allow
        ) {
            date
            volumeUSD
            tvlUSD
        }
    }
"""

In [16]:
fetch_pool_error = []


async def fetch_pool_chart_data(
    address: str,
    symbol0: str,
    symbol1: str,
    fee_tier: int,
    transport: AIOHTTPTransport = transport,
):
    error = False
    skip = 0
    all_found = False
    result = {"poolDayDatas": []}

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
        execute_timeout=20,
    ) as session:
        params = {"address": address, "startTime": START_TIMESTAMP, "skip": skip}
        try:
            while not all_found:
                temp = await session.execute(
                    gql(POOL_DAY_TIME_SERIES), variable_values=params
                )
                skip += 1000
                if len(temp["poolDayDatas"]) < 1000 or error:
                    all_found = True
                if temp:
                    # concat the lists
                    result["poolDayDatas"] = (
                        result["poolDayDatas"] + temp["poolDayDatas"]
                    )
        except Exception as e:
            print(e)
            error = True
            fetch_pool_error.append(address)

    if not error:
        with open(
            f"{DATA_POOL_DAY_PATH}/{get_pool_key(symbol0, symbol1, fee_tier)}.json",
            "w",
        ) as f:
            json.dump(result, f, indent=4)

In [17]:
fetch_pool_error = []

if refetch:
    # remove existing content in the out folder
    for f in glob.glob(DATA_POOL_DAY_PATH + "/*"):
        os.remove(f)

    # fetch pool data for each pool
    for i, row in pools_df.iterrows():
        await fetch_pool_chart_data(
            row["id"],
            row["token0.symbol"],
            row["token1.symbol"],
            row["feeTier"],
        )

    print(fetch_pool_error)

[]


In [18]:
# reads pool day datas from json
pool_day_df = pd.DataFrame(columns=["date"])
pool_names = []

for f in os.listdir(DATA_POOL_DAY_PATH):
    fullname = os.fsdecode(f)

    # not a rigorous check
    with open(os.path.join(DATA_POOL_DAY_PATH, fullname), "r") as file:
        pool_day_datas = json.load(file)

    # parse dict as df
    temp = pd.DataFrame.from_dict(pool_day_datas["poolDayDatas"]).astype(
        {"volumeUSD": np.float64, "tvlUSD": np.float64}
    )

    # prefix columns (except "date") with pool name
    cols = temp.columns[~temp.columns.isin(["date"])]
    pool_name = fullname.split(os.sep)[-1].split(".")[0]
    pool_names.append(pool_name)
    temp.rename(columns=dict(zip(cols, pool_name + "_" + cols)), inplace=True)

    # outer join: union of items on "date"
    pool_day_df = pd.merge(pool_day_df, temp, how="outer", on=["date"])

# sort by "date"
pool_day_df = pool_day_df.sort_values(by="date").reset_index(drop="index")

pool_day_df.head()

,date,BUSD_USDC_500_volumeUSD,BUSD_USDC_500_tvlUSD,DAI_USDC_100_volumeUSD,DAI_USDC_100_tvlUSD,DAI_USDC_500_volumeUSD,DAI_USDC_500_tvlUSD,DAI_WETH_3000_volumeUSD,DAI_WETH_3000_tvlUSD,USDC_USDT_100_volumeUSD,...,WBTC_USDC_3000_volumeUSD,WBTC_USDC_3000_tvlUSD,WBTC_WETH_3000_volumeUSD,WBTC_WETH_3000_tvlUSD,WBTC_WETH_500_volumeUSD,WBTC_WETH_500_tvlUSD,WETH_USDT_3000_volumeUSD,WETH_USDT_3000_tvlUSD,WETH_USDT_500_volumeUSD,WETH_USDT_500_tvlUSD
0,1620086400,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,...,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
1,1620172800,NaN,NaN,NaN,NaN,1.455737e+06,8.034837e+06,1.142416e+06,2.593409e+06,NaN,...,2.602096e+05,1.631491e+07,4.925324e+05,8.760155e+06,0.000000e+00,0.000000e+00,9.761890e+05,2.609716e+06,9.444342e+03,2.597984e+04
2,1620259200,NaN,NaN,NaN,NaN,1.519308e+07,1.732428e+07,2.054103e+07,1.558282e+07,NaN,...,2.844009e+06,1.590478e+07,7.507954e+06,3.459820e+07,1.102031e+03,5.146736e+03,2.672544e+07,2.649779e+07,2.357009e+05,5.846205e+04
3,1620345600,NaN,NaN,NaN,NaN,1.123453e+07,2.094898e+07,2.698736e+07,2.144639e+07,NaN,...,3.494689e+06,1.650661e+07,1.097052e+07,4.508025e+07,3.922264e+06,2.073400e+06,5.404438e+07,5.371375e+07,1.523177e+07,2.854816e+06
4,1620432000,NaN,NaN,NaN,NaN,1.691480e+07,2.929597e+07,1.786105e+07,2.063646e+07,NaN,...,4.087189e+06,1.710987e+07,2.327335e+07,5.307294e+07,7.443618e+06,1.980521e+06,9.324547e+07,5.172409e+07,2.939316e+07,2.159909e+06


In [19]:
# ["date"]: int -> date (in "YYYY-MM-DD")
pool_day_df["timestamp"] = pool_day_df["date"]  # keep timestamp in a new col
pool_day_df["date"] = pool_day_df["date"].map(dt.date.fromtimestamp)

pool_day_df.head()

,date,BUSD_USDC_500_volumeUSD,BUSD_USDC_500_tvlUSD,DAI_USDC_100_volumeUSD,DAI_USDC_100_tvlUSD,DAI_USDC_500_volumeUSD,DAI_USDC_500_tvlUSD,DAI_WETH_3000_volumeUSD,DAI_WETH_3000_tvlUSD,USDC_USDT_100_volumeUSD,...,WBTC_USDC_3000_tvlUSD,WBTC_WETH_3000_volumeUSD,WBTC_WETH_3000_tvlUSD,WBTC_WETH_500_volumeUSD,WBTC_WETH_500_tvlUSD,WETH_USDT_3000_volumeUSD,WETH_USDT_3000_tvlUSD,WETH_USDT_500_volumeUSD,WETH_USDT_500_tvlUSD,timestamp
0,2021-05-04,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,...,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,1620086400
1,2021-05-05,NaN,NaN,NaN,NaN,1.455737e+06,8.034837e+06,1.142416e+06,2.593409e+06,NaN,...,1.631491e+07,4.925324e+05,8.760155e+06,0.000000e+00,0.000000e+00,9.761890e+05,2.609716e+06,9.444342e+03,2.597984e+04,1620172800
2,2021-05-06,NaN,NaN,NaN,NaN,1.519308e+07,1.732428e+07,2.054103e+07,1.558282e+07,NaN,...,1.590478e+07,7.507954e+06,3.459820e+07,1.102031e+03,5.146736e+03,2.672544e+07,2.649779e+07,2.357009e+05,5.846205e+04,1620259200
3,2021-05-07,NaN,NaN,NaN,NaN,1.123453e+07,2.094898e+07,2.698736e+07,2.144639e+07,NaN,...,1.650661e+07,1.097052e+07,4.508025e+07,3.922264e+06,2.073400e+06,5.404438e+07,5.371375e+07,1.523177e+07,2.854816e+06,1620345600
4,2021-05-08,NaN,NaN,NaN,NaN,1.691480e+07,2.929597e+07,1.786105e+07,2.063646e+07,NaN,...,1.710987e+07,2.327335e+07,5.307294e+07,7.443618e+06,1.980521e+06,9.324547e+07,5.172409e+07,2.939316e+07,2.159909e+06,1620432000


In [20]:
# only consider pools with track record >= 180 days
pool_day_notna = pool_day_df.notna().sum()
cols_to_drop = list(pool_day_notna[pool_day_notna < 180].index.values)
pools_to_drop = list(set([x.rpartition("_")[0] for x in cols_to_drop]))

# drop cols
pools_df = pools_df[~pools_df["name"].isin(pools_to_drop)].reset_index()
pool_day_df.drop(columns=cols_to_drop, inplace=True)

print(pools_to_drop)

[]


In [21]:
pool_day_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       414 non-null    object 
 1   BUSD_USDC_500_volumeUSD    382 non-null    float64
 2   BUSD_USDC_500_tvlUSD       382 non-null    float64
 3   DAI_USDC_100_volumeUSD     221 non-null    float64
 4   DAI_USDC_100_tvlUSD        221 non-null    float64
 5   DAI_USDC_500_volumeUSD     414 non-null    float64
 6   DAI_USDC_500_tvlUSD        414 non-null    float64
 7   DAI_WETH_3000_volumeUSD    414 non-null    float64
 8   DAI_WETH_3000_tvlUSD       414 non-null    float64
 9   USDC_USDT_100_volumeUSD    221 non-null    float64
 10  USDC_USDT_100_tvlUSD       221 non-null    float64
 11  USDC_WETH_10000_volumeUSD  413 non-null    float64
 12  USDC_WETH_10000_tvlUSD     413 non-null    float64
 13  USDC_WETH_3000_volumeUSD   414 non-null    float64

In [22]:
# inspect number of days elapsed
d0 = pool_day_df["date"].iloc[0]
d1 = pool_day_df["date"].iloc[-1]
print(f"{d0} to {d1} has {(d1 - d0).days} days")

2021-05-04 to 2022-06-21 has 413 days


In [23]:
if refetch:
    pool_day_df.to_csv(os.path.join(DATA_PATH, "pool_day_df.csv"), index=False)